# **Question 2**

In [0]:
import numpy as np
import string
import os
import cv2
from sklearn.model_selection import train_test_split
from random import random
from sklearn.preprocessing import MinMaxScaler
# from sklearn.decomposition import PCA
from sklearn import preprocessing
import math

In [0]:
def flattern(data):
  data_f=[]
  for i in range(data.shape[0]):
    data_f.append(data[i].flatten())
  return np.array(data_f)

## **PCA from Q1**

In [0]:
import numpy as np
class PCA_:
    def __init__(self,C=100):
        self.C=C
    def transform(self,data):
        cov=np.cov(data.T)
        eig_val, eig_vec= np.linalg.eig(cov)
        idx = eig_val.argsort()[::-1]   
        eig_val = eig_val[idx]
        eig_vec = eig_vec[:,idx]
        m= np.real(eig_vec[:,0:self.C])
        XX=np.dot(data,m)
        return XX

## **Load data and apply PCA with 100 component and add Bias**

In [5]:
data=[]
label=[]
dim=(100,100)
file_path='/content/drive/My Drive/A3/dataset'
for _, _, files in os.walk(file_path):
  for ii in files:
    img=cv2.imread(file_path+"/"+ii,cv2.IMREAD_GRAYSCALE)
    resized = cv2.resize(img,dim, interpolation = cv2.INTER_AREA)
    data.append(resized)
    l=ii.split("_")
    label.append((l[0]))
data=np.array(data)
label=np.array(label)
data=flattern(data)
data=PCA_(100).transform(data)
one=np.ones(data.shape[0])
data=preprocessing.scale(data)
one=one.reshape([data.shape[0],1])
data=np.append(data,one,axis=1)
print(data.shape)
scaler = MinMaxScaler()

(520, 101)


## **Sigmoid and Cost function**

In [0]:
def sigmoid(Z):
  scaler.fit(Z)
  Z=scaler.transform(Z)
  return 1/(1+np.exp(-Z))
def loss(h,y):
  m=len(y)
  cost = 1 / m * np.sum(-y * np.log(h) - (1 - y) * np.log(1 - h))
  return cost

## **Split data into train and test (80% 20%)**

In [0]:
X_train, X_test,Y_train,Y_test = train_test_split(data,label,test_size=0.2,random_state=42)

## **Gradient Descent function**

In [0]:
def grad(theta,pre,label,X,al):
  alpha=al
  error=(pre-label)
  theta=theta - alpha * (np.dot((X.T),error)/label.shape[0])
  return theta

##**Logistic Regression(one vs all)**

In [79]:
classes=np.unique(Y_train)
d={}
for cl in classes:
  k=True
  #theta=np.random.rand(X_train.shape[1],1)
  theta=np.zeros([X_train.shape[1],1])
  #theta=np.ones([X_train.shape[1],1])
  Z = np.dot(X_train,theta)
  value=sigmoid(Z)
  ty=[]
  new_theta=0
  for i in Y_train:
    if i ==cl:
      ty.append(1)
    else:
      ty.append(0)
  ty=np.array(ty)
  ty=ty.reshape([len(Y_train),1])
  for i in range(5000):
    theta=grad(theta,value,ty,X_train,0.1)
    Z = np.dot(X_train,theta)
    value=sigmoid(Z)
    if k:
      cost=loss(value,ty)
      new_theta=theta.copy()
      k=False
    else:
      new=loss(value,ty)
      if(new > cost):
        break
      new_theta=theta.copy()
      # print("Cost = " + str(cost))
      cost=new
  d[cl]=new_theta
  print(str(cl) + " class done")

000 class done
001 class done
002 class done
003 class done
004 class done
005 class done
006 class done
007 class done


## **Classification report and Accuracy**

In [80]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
pre=[]
index=list(d.keys())
for i in X_test:
  v=[]
  for j in d.values():
    v.append(np.dot(i.T,j))
  pre.append(index[v.index(max(v))])
pre=np.array(pre)
print("Accuracy : ",accuracy_score(Y_test,pre))

print(classification_report(Y_test,pre,labels=np.unique(Y_test)))

from sklearn.metrics import confusion_matrix
print("\n\nConfusion_Matrix")
print(confusion_matrix(Y_test,pre))

Accuracy :  0.6442307692307693
              precision    recall  f1-score   support

         000       0.50      0.64      0.56        11
         001       0.80      0.53      0.64        15
         002       0.54      0.70      0.61        10
         003       0.67      0.57      0.62        14
         004       0.53      0.80      0.64        10
         005       0.91      0.77      0.83        13
         006       0.55      0.40      0.46        15
         007       0.72      0.81      0.76        16

    accuracy                           0.64       104
   macro avg       0.65      0.65      0.64       104
weighted avg       0.66      0.64      0.64       104



Confusion_Matrix
[[ 7  0  1  1  0  0  0  2]
 [ 4  8  2  0  0  0  1  0]
 [ 0  0  7  0  0  0  3  0]
 [ 0  0  0  8  2  1  0  3]
 [ 0  0  1  0  8  0  1  0]
 [ 1  1  0  1  0 10  0  0]
 [ 2  1  1  0  5  0  6  0]
 [ 0  0  1  2  0  0  0 13]]
